In [1]:
import cv2
import time
import src.viztools as vv
from __future__ import print_function

 ### TODO
 * set up patrol cycle option

In [126]:
reload(vv)

<module 'src.viztools' from 'src/viztools.py'>

In [ ]:
"""
Run vid-viz

KEYBOARD INPUTS:
    number keys - choose effect (see viztools.py for detailed keyboard input for each effect)
        0 - thresholding
        1 - alien
        3 - rgbwalk
        4 - rgbburst
    q - quit effect (then number keys are once again available for choosing a new effect)
    ` - enable toggling of border effects ('n' to toggle)
    esc - exit loop
"""


# general parameters
USE_CAM = 0
DISP_FULL_SCREEN = 0
LOOP = 1 # 1 to loop, 0 to patrol cycle

# select video source
if USE_CAM:
    cap = cv2.VideoCapture(0)
else:
#     cap = cv2.VideoCapture(0)
#     cap = cv2.VideoCapture('/media/data/Dropbox/Git/vid-viz/data/tunnel_01.mp4')
#     cap = cv2.VideoCapture('/home/mattw/Videos/2016-12-18 23-15-59.mp4')
#     cap = cv2.VideoCapture('/home/mattw/Videos/2016-08-26 02-48-16.mp4')
    cap = cv2.VideoCapture('/media/data/Dropbox/Git/vid-viz/data/snowflake_01.mp4')
    
# get video properties
# target_fps = cap.get(cv2.CAP_PROP_FPS)
target_fps = 20.0
tot_fr_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# initialize key handler
effect_index = None
post_process = False
key_list = [False for i in range(256)]

# initialize processing objects
effects = [vv.Threshold(),
           vv.Alien(),
           vv.RGBWalk(),
           vv.RGBBurst()]
num_effects = len(effects)
border = vv.Border()

fr_count = 0
while(cap.isOpened()):

    time_pre = time.time()
    
    # get frame and relevant info
    ret, frame = cap.read()
    
    if USE_CAM:
        # crop image to have same aspect ratio as monitor
        frame = frame[60:-60,:,:]

    # get keyboard input
    key = cv2.waitKey(1) & 0xFF
    key_list[key] = True
    if key == 27:
        # escape key
        break
    elif key == ord('q'):
        # quit current effect
        effect_index = None
    elif key == ord('`'):
        post_process = not post_process
        
    if effect_index is None:
        for num in range(10):
            if key == ord(str(num)):
                effect_index = num
                key_list[key] = False
        
    # process frame
    if effect_index is not None and effect_index < num_effects:
        frame = effects[effect_index].process(frame, key_list)
    
    # apply borders last to save processing time
    if post_process:
        frame = border.process(frame, key_list)

    # display frame
    if DISP_FULL_SCREEN:
        cv2.namedWindow('frame', cv2.WND_PROP_FULLSCREEN)
        cv2.setWindowProperty('frame', cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
        cv2.imshow('frame', frame)
    else:
        cv2.imshow('frame', frame)
        if fr_count == 0:
            cv2.moveWindow('frame', 0, 0)
    
    # reset video
    fr_count += 1
    if LOOP:
        if fr_count == tot_fr_count:
            # reset frame postion to zero
            cap.set(cv2.CAP_PROP_POS_FRAMES,1)
            fr_count = 1
    else:
        if fr_count == tot_fr_count:
            # reset frame postion to zero
            cap.set(cv2.CAP_PROP_POS_FRAMES,1)
            fr_count = 1        
    
    # calculate, limit and output fps
    time_tot = time.time() - time_pre
    if time_tot < 1/target_fps:
        time.sleep(1/target_fps - time_tot)
    time_tot = time.time() - time_pre
    print('\r%03i fps' % (1.0 / time_tot), end='')
    
    # reset key list (for pressed keys that are not reset by effect object(s))
    key_list[key] = False
    
cap.release()
cv2.destroyWindow('frame')

019 fps

In [119]:
%timeit cap.set(cv2.CAP_PROP_POS_FRAMES,1)

The slowest run took 133.82 times longer than the fastest. This could mean that an intermediate result is being cached.
10000000 loops, best of 3: 183 ns per loop


In [68]:
img1 = cv2.imread('/media/data/Dropbox/Git/vid-viz/data/test_img.png')

# img0 = 0
# img2 = cv2.addWeighted(img0, 0.1, img1, 0.9, 0)
# %timeit cv2.addWeighted(img0, 0.1, img1, 0.9, 0)

# im_width, im_height, _ = img1.shape
# %timeit cv2.warpAffine(img1, \
#             np.float32([[1, 0, 100], \
#                         [0, 1, 200]]), \
#             (im_width, im_height))

%timeit cv2.resize(img1, None, fx=2, fy=2, interpolation = cv2.INTER_LINEAR)

100 loops, best of 3: 2.51 ms per loop


In [20]:
import numpy as np
x = np.zeros(img0.shape, dytpe='uint8')

AttributeError: 'int' object has no attribute 'shape'

In [23]:
whos

Variable           Type                   Data/Info
---------------------------------------------------
DISP_FULL_SCREEN   int                    0
USE_CAM            int                    0
border             Border                 <src.viztools.Border object at 0x7f69035d8410>
cap                VideoCapture           <VideoCapture 0x7f68fc174fb0>
cv2                module                 <module 'cv2' from '/usr/<...>.7/dist-packages/cv2.so'>
effect_index       int                    3
effects            list                   n=4
fr_count           int                    79
frame              ndarray                540x960x3: 1555200 elems, type `uint8`, 1555200 bytes (1 Mb)
i                  int                    255
img                ndarray                938x935x3: 2631090 elems, type `uint8`, 2631090 bytes (2 Mb)
img0               int                    0
img1               ndarray                938x935x3: 2631090 elems, type `uint8`, 2631090 bytes (2 Mb)
img2           